# 이미지 설명 요청하기

## 인터넷에 있는 이미지 사진 설명 요청하기
- 원하는 이미지 링크 사용
- 단, API 호출을 통해 서버에 접근 가능한 이미지 링크를 사용하기

In [1]:
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("OPEN_API_KEY")  # 환경 변수에서 API 키를 가져오기

client = OpenAI(api_key=api_key)  # 오픈AI 클라이언트의 인스턴스 생성

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": "https://recipe1.ezmember.co.kr/cache/recipe/2016/06/05/1ba7e34cf0daf694f09a3a9539ebdb161.jpg",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response


ChatCompletion(id='chatcmpl-CQRZovVOTQnC6nZdCI8GZ1XKLIDhp', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='이 이미지는 오므라이스를 담고 있습니다. 오므라이스는 볶음밥을 얇은 달걀 지단으로 덮은 요리입니다. 달걀 위에 케첩이 뿌려져 있으며, 오므라이스에는 햄과 야채가 들어간 볶음밥이 보입니다. 접시에는 브로콜리와 얇게 썬 사과가 함께 곁들여져 있습니다. 옆에는 케첩이 곁들여져 있습니다.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[]))], created=1760418568, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_cbf1785567', usage=CompletionUsage(completion_tokens=117, prompt_tokens=779, total_tokens=896, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

## 내가 가진 이미지로 설명 요청하기

### base64 인코딩
- 바이너리 데이터 HTTP Request 전송을 위해 인코딩 진행

In [3]:
import base64

# Function to encode the image
def encode_image(path):
    with open(path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")
    
image_path = "./data/mart.jpg"

# 이미지를 base64로 인코딩
base64_image = encode_image(image_path)

print(base64_image[0:100])

/9j/4AAUSkZJRgABAQEBLAEsAABBTVBG/+EJxEV4aWYAAE1NACoAAAAIAA0BDwACAAAABgAAAKoBEAACAAAACgAAALABEgADAAAA


### GPT API 호출 및 결과 출력

In [4]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "이 이미지에 대해 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{base64_image}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'이미지에는 가게의 선반에 다양한 종류의 인스턴트 라면이 진열되어 있습니다. 왼쪽 상단에는 봉지 라면이 쌓여있고, 선반 위에는 컵라면들이 놓여 있습니다. 아래쪽에는 다른 식료품들이 보관되어 있는 모습입니다. 라면의 포장지에는 태국어가 적혀 있어 아시아 식료품점으로 보입니다.'

### 여러 이미지 비교 분석 요청

In [5]:
red_dumpling_base64 = encode_image("data/red_dumpling.jpeg")
dumpling_soup_base64 = encode_image("data/dumpling_soup.jpeg")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "두 음식의 차이점을 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{red_dumpling_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{dumpling_soup_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'두 음식의 차이점은 다음과 같습니다:\n\n1. **첫 번째 음식**:\n   - **소스**: 빨간 고추 기름 소스에 담겨 있습니다. 이로 인해 매콤한 맛이 날 수 있습니다.\n   - **조리법**: 비빔식으로, 만두가 소스와 함께 섞여 있습니다.\n   \n2. **두 번째 음식**:\n   - **국물**: 맑은 국물에 담겨 있으며, 국물 자체가 담백한 맛을 제공합니다.\n   - **첨가물**: 고명으로 얇게 썬 지단과 유부 등이 들어 있습니다.\n   - **조리법**: 국물 요리로, 만두탕이나 만두국 형태입니다.\n\n이로 인해 첫 번째 음식은 좀 더 자극적이고 두 번째 음식은 담백한 맛을 즐길 수 있는 차이가 있습니다.'

## GPT 비전의 한계 알아보기

In [6]:
oecd_rnd_2021_base64 = encode_image("./data/oecd_rnd_2021_large.png")
oecd_rnd_2022_base64 = encode_image("./data/oecd_rnd_2022_large.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content


'두 이미지는 주로 국가별 연구개발비(R&D)와 GDP 대비 연구개발비 비중을 비교하고 있습니다. 한국을 중심으로 2021년과 2022년 데이터의 변화를 설명하면 다음과 같습니다:\n\n1. **연구개발비(백만 USD)**:\n   - 2021년: 89,282백만 USD\n   - 2022년: 91,013백만 USD\n   \n   한국의 연구개발비는 2022년에 약간 증가했습니다.\n\n2. **GDP 대비 연구개발비 비중(%)**:\n   - 2021년: 4.93%\n   - 2022년: 4.81%\n   \n   한국의 GDP 대비 연구개발비 비중은 2021년에 비해 2022년에 약간 감소했습니다.\n\n3. **비교 분석**:\n   - 연구개발비의 절대값은 증가했지만, GDP 대비 비중은 약간 감소하여, GDP 증가에 비해 연구개발비 증가는 상대적으로 더 적었던 것으로 보입니다.\n   - 다른 국가와 비교했을 때도, 한국은 여전히 높은 연구개발비 비중을 유지하고 있습니다. 이는 한국의 강한 연구개발 투자 의지를 보여줍니다.\n\n이러한 데이터는 한국이 여전히 글로벌 과학 기술 분야에서 경쟁력을 유지하고자 하는 노력을 하고 있음을 나타냅니다.'

In [7]:
oecd_rnd_2021_base64 = encode_image("./data/oecd_rnd_2021_medium.png")
oecd_rnd_2022_base64 = encode_image("./data/oecd_rnd_2022.png")

messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "첫번째는 2021년 데이터이고, 두번째는 2022년 데이터입니다. 이 데이터에 대해 설명해주세요. 어떤 변화가 있었나요? 한국 중심으로 설명해주세요."},
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2021_base64}",
                },
            },
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{oecd_rnd_2022_base64}",
                },
            },
        ],
    }
]

response = client.chat.completions.create(
    model="gpt-4o",  # 응답 생성에 사용할 모델 지정
    messages=messages # 대화 기록을 입력으로 전달
)

response.choices[0].message.content

'두 이미지를 비교하여 한국의 연구개발비의 변화를 분석해보겠습니다.\n\n1. **연구개발비 변화:**\n   - 2021년: $121,739백만 (약 1,217억 3,900만 달러)\n   - 2022년: $87,225백만 (약 872억 2,500만 달러)\n\n   한국의 연구개발비가 2021년에서 2022년 사이에 감소했습니다.\n\n2. **GDP 대비 연구개발비 비중:**\n   - 2021년: 4.93%\n   - 2022년: 4.81%\n\n   비중이 약간 감소했음을 알 수 있습니다.\n\n3. **다른 국가와의 비교:**\n   - **미국:** 연구개발비가 증가 (약 8% 증가)\n   - **중국:** 연구개발비가 소폭 증가\n\n이런 변화를 통해 한국은 연구개발에 대한 투입을 전년 대비 줄였으며, GDP 대비 연구개발 투자 비율도 소폭 감소했습니다. 이는 다른 주요국과 비교했을 때 상대적으로 큰 변화로 보입니다. 경제 상황이나 정책 변화 등에 따른 결과일 수 있으므로 다양한 요인을 고려해 분석할 필요가 있습니다.'